In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/gdrive/MyDrive/kaggle_datasets'

In [ ]:
%cd /content/gdrive/MyDrive/kaggle_datasets

/content/gdrive/MyDrive/kaggle_datasets


In [ ]:
!ls

chest_xray  kaggle.json


In [ ]:
#!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
#!unzip chest-xray-pneumonia.zip

In [ ]:
import numpy as np 
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras as keras
from keras.layers import Dropout


In [ ]:
DIRECTORY = r'/content/gdrive/MyDrive/kaggle_datasets/chest_xray'
FOLDERS = ['test','train', 'val']

CLASS = ['NORMAL','PNEUMONIA']


In [ ]:
IMAGE_SIZE = (224, 224, 3) 

def preprocess_data(train, test):
    generator = keras.preprocessing.image.ImageDataGenerator()

    train_dataset = generator.flow_from_directory(batch_size = 32,
    directory = train,
    shuffle = True,
    target_size = IMAGE_SIZE[:2],
    class_mode = 'binary'
    )

    val_dataset = generator.flow_from_directory(batch_size = 32,
    directory = test,
    shuffle = True,
    target_size = IMAGE_SIZE[:2],
    class_mode = 'binary'
    )
    return train_dataset, val_dataset


In [ ]:
train_generator, test_generator = preprocess_data('/content/gdrive/MyDrive/kaggle_datasets/chest_xray/train',
                                                  '/content/gdrive/MyDrive/kaggle_datasets/chest_xray/test')


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.EfficientNetB0(
                                            include_top = False,
                                            weights = 'imagenet')


In [ ]:
base_model.trainable = False
inputs = keras.Input(shape=IMAGE_SIZE)  
x = base_model(inputs) 
x = keras.layers.Conv2D(128, 3, activation = 'relu')(x) 
x = keras.layers.GlobalAveragePooling2D()(x) 
x = keras.layers.Dropout(.2)(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

In [ ]:
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 128)         1474688   
_________________________________________________________________
global_average_pooling2d (Gl (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 5,524,388
Trainable params: 1,474,817
Non-trainable params: 4,049,571
___________________________________________

In [ ]:
model.compile(optimizer = keras.optimizers.Adam(), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(train_generator, epochs = 2, validation_data = test_generator)

Epoch 1/2
163/163 [==============================] - ETA: 0s - loss: 0.4011 - accuracy: 0.8760

In [ ]:
model.evaluate(test_generator)

In [ ]:
model.save('PneuNetV2.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
with open('PneuNetV2_Lite.tflite', 'wb') as f:
  f.write(tflite_model)